In [1]:
! pip install transformers 
! pip install datasets 
! pip install huggingface_hub

In [2]:
# !hugging_face --version

/bin/bash: hugging_face: command not found


In [3]:
# from huggingface_hub import notebook_login

# notebook_login()

In [6]:
# !apt install git-lfs
# !git config --global user.email "patelkrinal214@gmail.com"
# !git config --global user.name "krinal214"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 75 not upgraded.


In [7]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [8]:
import transformers
print(transformers.__version__)

4.16.2


In [9]:
model_checkpoint = "bert-base-multilingual-cased"
batch_size = 16

In [10]:
from datasets import load_dataset, load_metric


In [12]:
dataset1=load_dataset('squad')

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
datasets=load_dataset("tydiqa","secondary_task")
datasets

Downloading:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/161M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset tydiqa downloaded and prepared to /root/.cache/huggingface/datasets/tydiqa/secondary_task/1.0.0/b8a6c4c0db10bf5703d7b36645e5dbae821b8c0e902dac9daeecd459a8337148. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 49881
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5077
    })
})

In [13]:
import re
def filter_dataset(ds,lang):
    def check(d):
        for i in lang:
            if re.search(i,d):
              return True
        return False
    ds = ds.filter(lambda x: check(x['id']))
    return ds

In [14]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [15]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=8):
    picks = random.sample(range(0, len(dataset)-1), num_examples)
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [16]:
from datasets import concatenate_datasets
final_dataset = concatenate_datasets([dataset1['train'],datasets['train']])
final_dataset= concatenate_datasets([final_dataset,dataset1['validation']])

In [17]:
datasets["train"]=final_dataset
datasets["validation"]=filter_dataset(datasets["validation"],['telugu','bengali'])

  0%|          | 0/6 [00:00<?, ?ba/s]

In [18]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 148050
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 782
    })
})


In [19]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,570d6bb6b3d812140066d8c5,Valencia,"Balansiyya had a rebirth of sorts with the beginning of the Taifa of Valencia kingdom in the 11th century. The town grew, and during the reign of Abd al-Aziz a new city wall was built, remains of which are preserved throughout the Old City (Ciutat Vella) today. The Castilian nobleman Rodrigo Diaz de Vivar, known as El Cid, who was intent on possessing his own principality on the Mediterranean, entered the province in command of a combined Christian and Moorish army and besieged the city beginning in 1092. By the time the siege ended in May 1094, he had carved out his own fiefdom—which he ruled from 15 June 1094 to July 1099. This victory was immortalised in the Lay of the Cid. During his rule, he converted nine mosques into churches and installed the French monk Jérôme as bishop of the See of Valencia. El Cid was killed in July 1099 while defending the city from an Almoravid siege, whereupon his wife Ximena Díaz ruled in his place for two years.",Who was the ruler when the new city wall was constructed?,"{'text': ['Abd al-Aziz'], 'answer_start': [146]}"
1,arabic--3198255694329407591-0,أمل كلوني,أمل علم الدين (اسم الولادة أمل علم الدين، مولودة في 3 فبراير 1978) هي محامية وناشطة حقوقية وكاتبة إنجليزية من أصل لبناني. تعمل محاميةً في مكتب دوتي ستريت تشامبرز، متخصصة في القانون الدولي وحقوق الإنسان.[1].,متى ولدت أمل علم الدين المحامية الناشطة الحقوقية والكاتبة الإنجليزية ؟,"{'text': ['3 فبراير 1978'], 'answer_start': [53]}"
2,korean--4765368616512639259-0,플레이스테이션 4,"플레이스테이션 4(, , PS4)는 2013년에 출시된 가정용 게임기이다. 줄여서 PS4라고도 한다. 2013년 2월 20일(동부 표준시)에 뉴욕에서 이루어진 이벤트 '플레이스테이션 미팅 2013'(PlayStation Meeting 2013)에서 소니가 발표하였고[1] 11월 15일, 북미에서 처음 출시되어 출시 24시간만에 100만대 이상 판매하는 기록을 세웠다.[2] 2014년 3월 2일 누적 판매량 600만대 이상을 넘었다.",플레이스테이션 4 개발사는 어디인가요?,"{'text': ['소니'], 'answer_start': [139]}"
3,arabic-8033925895297205775-0,ويليام هيرشل,فريدريك ويليام هيرشل (15 نوفمبر 1738 – 25 أغسطس 1822) عالم فلك وملحن بريطاني من أصل ألماني. اتبع هيرشل خطوات أبيه فانضم للفرقة الموسيقية العسكرية التابعة لهانوفر، قبل هجرته إلى بريطانيا العظمى عن عمر يُناهز التاسع عشر، وهو شقيق العالِمة كارولين هيرشل. أصبح مشهوراً لاكتشافه كوكب أورانوس، وقمريه الرئيسيين تيتانيا، وأوبيرون، بالإضافة إلى قمري زحل. وقد كان أيضا أول من اكتشف وجود الأشعة تحت الحمراء. وألف وإشتُهر بسمفونياته الأربعة والعشرين، وعدداً من القطع الموسيقية التي لحنها.,من هو ويليام هيرشل؟,"{'text': ['عالم فلك وملحن بريطاني من أصل ألماني'], 'answer_start': [55]}"
4,572672e8708984140094c679,Dutch_language,"Both languages are still largely mutually intelligible, although this relation can in some fields (such as lexicon, spelling and grammar) be asymmetric, as it is easier for Dutch speakers to understand written Afrikaans than it is for Afrikaans speakers to understand written Dutch. Afrikaans is grammatically far less complex than Dutch, and vocabulary items are generally altered in a clearly patterned manner, e.g. vogel becomes voël (""bird"") and regen becomes reën (""rain""). In South Africa, the number of students following Dutch at university, is difficult to estimate, since the academic study of Afrikaans inevitably includes the study of Dutch. Elsewhere in the world, the number of people learning Dutch is relatively small.","What's the Dutch word for ""bird""?","{'text': ['vogel'], 'answer_start': [418]}"
5,arabic--7161691222941897467-0,حركة الشباب الملكي,حركة الشباب الملكي أو الشماكرية: هي حركة موالية لنضام القائم بالمغرب ، تنظم مسيرات مضادة للمعارضة المغربية، خصوصا ضد حركة 20 فبراير.[1][2][3] يترأس الحركة المدعو عبد الصمد أوسياح.[4] سجل لها العديد من الاعتدائات على المتظاهرين[5][6][7],ما هي حركة الشباب الملكي أو الشماكرية؟,"{'text': ['حركة موالية لنضام القائم بالمغرب ، تنظم مسيرات مضادة للمعارضة المغربية، خصوصا ضد حركة 20 فبراير'], 'answer_start': [36]}"
6,56beb7953aeaaa14008c92ae,Super_Bowl_50,"The Broncos defeated the Pittsburgh Steelers in the divisional round, 23–16, by scoring 11 points in the final three minutes of the game. They then b

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [21]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 160  # The allowed overlap between two part of the context when splitting is performed.

In [22]:
def prepare_train_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1 :
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1
    
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [23]:
tokenized_datasets = datasets.map(
    prepare_train_features, batched=True, remove_columns=datasets["train"].column_names
)

  0%|          | 0/149 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-bas

In [25]:
from transformers import TrainingArguments, Trainer
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-model",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    # push_to_hub = True,
    # push_to_hub_model_id = "augmented",
    save_steps=9780,

)


Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.7/site-packages/transformers/training_args.py:951: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case krinal214/augmented).
  FutureWarning,


In [26]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [27]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning https://huggingface.co/krinal214/augmented into local empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [28]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 156586
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 930
    })
})


In [29]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 156586
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 9787


Epoch,Training Loss,Validation Loss
1,1.060900,0.510443


Saving model checkpoint to bert-base-multilingual-cased-finetuned-model/checkpoint-9780
Configuration saved in bert-base-multilingual-cased-finetuned-model/checkpoint-9780/config.json
Model weights saved in bert-base-multilingual-cased-finetuned-model/checkpoint-9780/pytorch_model.bin
tokenizer config file saved in bert-base-multilingual-cased-finetuned-model/checkpoint-9780/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased-finetuned-model/checkpoint-9780/special_tokens_map.json
tokenizer config file saved in bert-base-multilingual-cased-finetuned-model/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased-finetuned-model/special_tokens_map.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

***** Running Evaluation *****
  Num examples = 930
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=9787, training_loss=1.232053431449244, metrics={'train_runtime': 6612.1242, 'train_samples_per_second': 23.682, 'train_steps_per_second': 1.48, 'total_flos': 3.068656046269747e+16, 'train_loss': 1.232053431449244, 'epoch': 1.0})

In [30]:
def prepare_validation_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]

    
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [31]:
max_answer_length = 50
n_best_answers=20

In [32]:
from tqdm.auto import tqdm
import numpy as np

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 50):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    predictions = collections.OrderedDict()
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or len(offset_mapping[start_index])==0
                        or len(offset_mapping[end_index])==0
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    try:
                        start_char = offset_mapping[start_index][0]
                        end_char = offset_mapping[end_index][1]
                        valid_answers.append(
                            {
                                "score": start_logits[start_index] + end_logits[end_index],
                                "text": context[start_char: end_char]
                            }
                        )
                    except IndexError:
                        continue

        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["id"]] = best_answer["text"]
        
    return predictions


In [33]:
import collections
def evaluate(ds,wrong_pred,language):
    val_dataset=filter_dataset(ds,language)
    validation_features = val_dataset.map(
        prepare_validation_features,
        batched=True,
        remove_columns=val_dataset.column_names
    )
    raw_predictions = trainer.predict(validation_features)
    validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))
    
    examples = val_dataset
    features = validation_features

    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    final_predictions = postprocess_qa_predictions(val_dataset, validation_features, raw_predictions.predictions,n_best_answers,max_answer_length)
    metric = load_metric("squad")
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in val_dataset]
    for j in range(len(references)):
        wrong=True
        for t in references[j]["answers"]["text"]:
            if normalize_answer(formatted_predictions[j]['prediction_text'])==normalize_answer(t):
                wrong=False
                break
        if wrong:
            wrong_pred.append({"id":references[j]["id"],"prediction":formatted_predictions[j]["prediction_text"],"original:":t})
    return metric.compute(predictions=formatted_predictions, references=references)

In [34]:
import string
eval_languages = [['bengali'], ['telugu']]
wrong_prediction={}
wrong_prediction['bengali']=[]
wrong_prediction['telugu']=[]
wrong_prediction['all']=[]
for lang in eval_languages:
    output = evaluate(datasets["validation"],wrong_prediction[lang[0]],lang)
    print(lang[0],':',output)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 144
  Batch size = 16


Post-processing 113 example predictions split into 144 features.


  0%|          | 0/113 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 786
  Batch size = 16


Post-processing 669 example predictions split into 786 features.


  0%|          | 0/669 [00:00<?, ?it/s]

telugu : {'exact_match': 71.74887892376681, 'f1': 83.98478950048458}


In [35]:
output = evaluate(datasets["validation"],wrong_prediction['all'],['telugu','bengali'])
print('overall',output)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 930
  Batch size = 16


Post-processing 782 example predictions split into 930 features.


  0%|          | 0/782 [00:00<?, ?it/s]

overall {'exact_match': 70.84398976982096, 'f1': 83.05181058238995}


In [36]:
print('wrong predictions(bengali)',len(wrong_prediction['bengali']))
print('wrong predictions(telugu)',len(wrong_prediction['telugu']))


wrong predictions(bengali) 39
wrong predictions(telugu) 189


In [39]:
picks = random.sample(range(0, len(wrong_prediction['bengali'])-1), 38)
for i in picks:
    print(wrong_prediction['bengali'][i])

{'id': 'bengali--5743963435064074650-0', 'prediction': '১৯৭৫', 'original:': '১৯৭৫ সালে'}
{'id': 'bengali--2217820890808407965-4', 'prediction': '১৯৮৬ সালের ২৯ এপ্রিল', 'original:': '১৯৮৪ সালের ২৪ অক্টোবর'}
{'id': 'bengali--7369128730344741991-1', 'prediction': 'মির্জা মুহম্মদ বেগ', 'original:': 'আরব'}
{'id': 'bengali-5730130221372563642-10', 'prediction': '২০১৬ সালের ২৩ জুলাই হৃদরোগে আক্রান্ত হয়ে মহাশ্বেতা দেবী কলকাতার বেল ভিউ ক্লিনিকে ভর্তি হন। সেই বছরই ২৮ জুলাই', 'original:': '২০১৬ সালের ২৩ জুলাই'}
{'id': 'bengali--1366347128155871013-24', 'prediction': 'রাজা বরেণ্য', 'original:': 'কার্তিকেয়'}
{'id': 'bengali--8422112448971508966-2', 'prediction': '১০৫টি', 'original:': '১০৫'}
{'id': 'bengali-94515828498152131-10', 'prediction': '২.৪ মিলিয়ন বছর', 'original:': '৫০০,০০০'}
{'id': 'bengali-599666913917674019-13', 'prediction': 'দানব বংশের মহিরঙ্গ', 'original:': 'মহিরঙ্গ'}
{'id': 'bengali-4841554072514664155-0', 'prediction': 'ইন্দিরা গান্ধী', 'original:': 'শেখ মুজিবুর রহমান'}
{'id': 'b

In [40]:
picks = random.sample(range(0, len(wrong_prediction['telugu'])-1), 40)
for i in picks:
    print(wrong_prediction['telugu'][i])

{'id': 'telugu-32885819921748529-1', 'prediction': 'జమ్ము మరియు కాశ్మీర్ లోని ఫగ్వాల్\u200cలో', 'original:': 'జమ్ము మరియు కాశ్మీర్ లోని ఫగ్వాల్\u200c'}
{'id': 'telugu-8243050460142458655-0', 'prediction': '1961 హెక్టార్లలో', 'original:': '1961 హెక్టార్ల'}
{'id': 'telugu--8618704549090519564-0', 'prediction': 'కొమరం చిన్నూ-సోంబారు', 'original:': 'సోంబారు'}
{'id': 'telugu-5459876262173192429-0', 'prediction': '149 హెక్టార్లలో', 'original:': '149 హెక్టార్ల'}
{'id': 'telugu-9115912749184044385-1', 'prediction': 'పశ్చిమ ఆఫ్ఘనిస్థాన్', 'original:': 'పశ్చిమ ఆఫ్ఘనిస్థాన్\u200c'}
{'id': 'telugu--7211034341454150780-1', 'prediction': 'విశాఖపట్నం', 'original:': 'విశాఖపట్నంలో'}
{'id': 'telugu-261437164357885089-0', 'prediction': 'కర్ణాటక రాష్ట్రములో కోలారు సమీపాన గల నందిదుర్గ కొండలలోని చెన్నకేశవ కొండల్లో', 'original:': 'ర్ణాటక రాష్ట్రములో కోలారు సమీపాన గల నందిదుర్గ కొండలలోని చెన్నకేశవ కొండల్లో'}
{'id': 'telugu--1526336483193941112-0', 'prediction': '3365 హెక్టార్లలో', 'original:': '3365 హెక్టార్ల'

In [41]:
# trainer.push_to_hub()

Saving model checkpoint to bert-base-multilingual-cased-finetuned-model
Configuration saved in bert-base-multilingual-cased-finetuned-model/config.json
Model weights saved in bert-base-multilingual-cased-finetuned-model/pytorch_model.bin
tokenizer config file saved in bert-base-multilingual-cased-finetuned-model/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased-finetuned-model/special_tokens_map.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Upload file pytorch_model.bin:   0%|          | 32.0k/676M [00:00<?, ?B/s]

Upload file runs/Mar29_15-02-25_9effcbfd2cf9/events.out.tfevents.1648566179.9effcbfd2cf9.60.0: 100%|##########…

To https://huggingface.co/krinal214/augmented
   0429e7d..e394e88  main -> main



...

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

To https://huggingface.co/krinal214/augmented
   e394e88..2b785ad  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'https://huggingface.co/krinal214/augmented/commit/e394e88cdc1d23a4df62bcb5dbf1e14329b9fe53'